네이버 Openapi와 urllib, beautifulsoup을 활용해 네이버를 검색했을 때
블로그 탭에 노출되는 게시물의 제목, 작성자, 날짜, 내용을 크롤링합니다.

In [11]:
import json
import math
import re
from datetime import datetime
import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from numpy import nan as NA
from pandas import Series,DataFrame

In [12]:
naver_client_id = "your_id"
naver_client_secret = "your_secret"

def naver_blog_crawling(search_blog_keyword, display_count, sort_type):
    search_result_blog_page_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)[0]
    total_posting_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)[1]
    crawled_dataframe = get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type)
    return crawled_dataframe

def get_blog_search_result_pagination_count(search_blog_keyword, display_count):
    encode_search_keyword = urllib.parse.quote(search_blog_keyword)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_keyword
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id", naver_client_id)
    request.add_header("X-Naver-Client-Secret", naver_client_secret)

    response = urllib.request.urlopen(request)
    response_code = response.getcode()

    if response_code is 200:
        response_body = response.read()
        response_body_dict = json.loads(response_body.decode('utf-8'))

        if response_body_dict['total'] == 0:
            blog_pagination_count = 0
        else:
            blog_pagination_total_count = math.ceil(response_body_dict['total'] / int(display_count))

            if blog_pagination_total_count >= 1000:
                blog_pagination_count = 1000
            else:
                blog_pagination_count = blog_pagination_total_count

        return blog_pagination_count,response_body_dict['total']
    
def get_blog_post(search_blog_keyword, display_count, search_result_blog_page_count, sort_type):
    encode_search_blog_keyword = urllib.parse.quote(search_blog_keyword)

    total_posting_count = get_blog_search_result_pagination_count(search_blog_keyword, display_count)[1]
    title,postdate,bloggername,text = [],[],[],[]

    for i in range(1, 101, display_count):
        url = "https://openapi.naver.com/v1/search/blog?query=" + encode_search_blog_keyword + "&display=" + str(
            display_count) + "&start=" + str(i) + "&sort=" + sort_type

        request = urllib.request.Request(url)

        request.add_header("X-Naver-Client-Id", naver_client_id)
        request.add_header("X-Naver-Client-Secret", naver_client_secret)

        response = urllib.request.urlopen(request)
        response_code = response.getcode()

        if response_code is 200:
            response_body = response.read()
            response_body_dict = json.loads(response_body.decode('utf-8'))

            for j in range(0, len(response_body_dict['items'])):
                blog_post_url = response_body_dict['items'][j]['link'].replace("amp;", "")

                get_blog_post_content_code = requests.get(blog_post_url)
                get_blog_post_content_text = get_blog_post_content_code.text

                get_blog_post_content_soup = BeautifulSoup(get_blog_post_content_text, 'lxml')
                
                df_for_row = pd.DataFrame()

                for link in get_blog_post_content_soup.select('#mainFrame'):
                    real_blog_post_url = "http://blog.naver.com" + link.get('src')

                    get_real_blog_post_content_code = requests.get(real_blog_post_url)
                    get_real_blog_post_content_text = get_real_blog_post_content_code.text

                    get_real_blog_post_content_soup = BeautifulSoup(get_real_blog_post_content_text, 'lxml')

                    remove_html_tag = re.compile('<.*?>')
                    
                    blog_post_title = re.sub(remove_html_tag, '', response_body_dict['items'][j]['title'])
                    blog_post_description = re.sub(remove_html_tag, '',
                                                   response_body_dict['items'][j]['description'])
                    blog_post_postdate = datetime.strptime(response_body_dict['items'][j]['postdate'],
                                                                    "%Y%m%d").strftime("%y.%m.%d")
                    blog_post_blogger_name = response_body_dict['items'][j]['bloggername']
                    
                    blog_post_content_text=""
                    
                    for blog_post_content in get_real_blog_post_content_soup.select('.se_textView'):
                        blog_post_content_text = blog_post_content_text + blog_post_content.get_text()
                    
                    title.append(blog_post_title)
                    postdate.append(blog_post_postdate)
                    bloggername.append(blog_post_blogger_name)
                    text.append(blog_post_content_text)
                    
    return pd.DataFrame({'title' : title, 'postdate' : postdate, 'bloggername' : bloggername, 'text' : text})

아래와 같이 검색어, 크롤링할 페이지수, 검색결과 정렬순서를 지정해 데이터프레임 형식으로 추출할 수 있습니다.

In [13]:
df = naver_blog_crawling('웹 크롤링', 10, 'sim')

In [14]:
df

,title,postdate,bloggername,text
0,"[Python]파이썬 크롤링(Crawling)/ 네이버, 멜론 웹크롤링",19.08.14,어준이 이야기,
1,[프로그래밍 입문반] 전수진님 : 웹 크롤링을 비전공자인 제가...,19.07.18,DS스쿨 공식블로그,
2,파이썬으로 배우는 초보 웹크롤링...,18.02.03,etilelab,\n\n파이썬으로 배우는 초보 웹크롤링(Beautifulsoup라이브러리 설치)\n...
3,웹 구조를 이해한 자의 웹크롤링은 데이터를 다루는...,19.07.15,패스트캠퍼스 : FAST CAMPUS,
4,파이선 웹크롤링 초보 탈출기 1,18.11.17,아름다운 도시를 향한 발걸음,\n\n파이선 웹크롤링 초보 탈출기 1\n\n\n 하루종일 컴퓨터 앞에 붙어서서 이...
5,뷰티풀 수프 이용해 보기 1부 : 웹 크롤링 개념,18.11.25,Data Miner,\n\n파이썬(Python) - Beautifulsoup 뷰티풀 수프 이용해 보기 ...
6,웹크롤링 [금통위의사록 파이썬으로 다운받기],19.07.13,Daikoku's blog,
7,[Python] 파이썬 웹 크롤링 #1. 네이버 실시간 검색어 가져오기,19.07.16,넬티아의 컴퓨터 블로그,
8,[파이썬으로 개발 시작하기 CAMP] - 파이썬 웹 크롤링,18.08.23,연시루 블로그,\n\n[파이썬으로 개발 시작하기 CAMP] - 파이썬 웹 크롤링\n\n\n※ 패스...
9,[웹크롤링] 부동산 데이터 크롤링 : 트룰리아 (trulia.com),19.07.10,웹 크롤링 & 웹 스크래핑,
